In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import ipaddress
from user_agents import parse
import httpagentparser

In [2]:
current_dataset = pd.read_csv("../data-processed/imputation-v2.csv")

In [4]:
current_dataset.head()

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERZIPCODE,PLATFORMTYPE,PLATFORM,BOT,BROWSER,DIST
0,1001,0,2,2863464470,6974dcaa-f932-480e-9fb5-c52e20e1393a,97206,3,Windows,False,Chrome,NaN
1,1002,0,2,1104739609,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,22202,3,Mac OS,False,Safari,NaN
2,1003,0,2,1122520854,a698de4b-e200-46dd-b5fb-40402175ae18,83654,3,Mac OS,False,Safari,NaN
3,1004,0,3,2302049782,45967533-75c8-4fbd-a00c-e6ff20447aaa,229114624,3,Windows,False,ChromiumEdge,NaN
4,1005,1,7,2932533091,a17e25be-532d-4cf5-b916-9308c8c3961f,77008,3,iOS,False,Safari,iPhone


In [5]:
dataset = current_dataset[~current_dataset["USERPLATFORMUID"].duplicated(keep=False)]

In [6]:
dataset

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERZIPCODE,PLATFORMTYPE,PLATFORM,BOT,BROWSER,DIST
1,1002,0,2,1104739609,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,22202,3,Mac OS,False,Safari,NaN
2,1003,0,2,1122520854,a698de4b-e200-46dd-b5fb-40402175ae18,83654,3,Mac OS,False,Safari,NaN
3,1004,0,3,2302049782,45967533-75c8-4fbd-a00c-e6ff20447aaa,229114624,3,Windows,False,ChromiumEdge,NaN
7,1008,1,2,404498651,1e84b963-2a2a-4e1a-a767-33867aa89bd4,75204,3,iOS,False,Safari,iPhone
8,1009,1,2,1756132554,be61a8f7-0f00-487f-8a56-679c819fa01a,91344,3,iOS,False,Safari,iPhone
...,...,...,...,...,...,...,...,...,...,...,...
109332,114932,1,2,800703282,eb251ab5-b64a-4022-a7fa-6f780119a653,75074,3,iOS,False,Safari,iPhone
109333,114933,0,2,1146249598,051e4a06-4cb4-4b97-b0b1-d1c6a2b8e2c2,19120,3,Mac OS,False,Safari,NaN
109335,114935,0,7,2920636017,9c36e90c-caa7-4323-8431-fb84f8293972,98105,3,Mac OS,False,Chrome,NaN
109336,114936,1,2,1174241667,afc27760-31f7-4f6e-aa08-90d8a96525d0,19808,3,iOS,False,Safari,iPhone


In [7]:
past_dataset = pd.read_csv("../data-processed/imputation-v1.csv")

In [9]:
new_past_dataset = past_dataset[~past_dataset["USERPLATFORMUID"].duplicated(keep=False)]

In [10]:
new_past_dataset

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
1,1002,Desktop,2,65.216.253.25,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,Arlington,22202.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/deci...,Bone Marrow|Radiography|Chronic|Oncology|Psych...,NaN,0.0
2,1003,Desktop,2,66.232.79.22,a698de4b-e200-46dd-b5fb-40402175ae18,New Meadows,83654.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
3,1004,Desktop,3,137.54.125.246,45967533-75c8-4fbd-a00c-e6ff20447aaa,NaN,229114624.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://globalrph.com/medcalcs/warfarin-mainte...,Dental|Total|Clinical|Pharmacology|Physicians|...,2084P0800X,1.0
7,1008,Mobile,2,24.28.40.219,1e84b963-2a2a-4e1a-a767-33867aa89bd4,Dallas,75204.0,Mozilla/5.0 (iPhone; CPU iPhone OS 16_3 like M...,Online Medical Journal,Website,https://www.clinicaladvisor.com/home/features/...,Family Practice|Drainage|Clinical|Dermatology|...,NaN,0.0
8,1009,Mobile,2,104.172.116.202,be61a8f7-0f00-487f-8a56-679c819fa01a,Granada Hills,91344.0,Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like M...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/deci...,Intensive Care|Shock|Ischemic Heart Disease|He...,2084P0804X,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113931,114932,Mobile,2,47.185.195.50,eb251ab5-b64a-4022-a7fa-6f780119a653,Plano,75074.0,Mozilla/5.0 (iPhone; CPU iPhone OS 16_3 like M...,Online Medical Journal,Website,https://www.rheumatologyadvisor.com/home/gener...,Myopathies|Small|Rheumatoid|Psychiatry|Urology...,NaN,0.0
113932,114933,Desktop,2,68.82.97.126,051e4a06-4cb4-4b97-b0b1-d1c6a2b8e2c2,Philadelphia,19120.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,Online Medical Journal,Website,https://www.clinicaladvisor.com/home/consultat...,Cardiovascular|Clinical|False|Male|Oncology|Ps...,207RP1001X,1.0
113934,114935,Desktop,7,174.21.94.113,9c36e90c-caa7-4323-8431-fb84f8293972,Seattle,98105.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cureus.com/articles/39455-whats-th...,Rehabilitation|Transplantation|Clinical|Preven...,208000000X,1.0
113935,114936,Mobile,2,69.253.129.131,afc27760-31f7-4f6e-aa08-90d8a96525d0,Wilmington,19808.0,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...,Online Medical Journal,Website,https://www.dermatologyadvisor.com/home/topics...,pain|disease|autoimmune|multiple sclerosis|gra...,2084P0800X,1.0
